In [38]:
import pandas
import aql
conn = aql.Connection("http://arachne.compbio.ohsu.edu")
O = conn.graph("bmeg")

In [41]:
c = O.query().V().where(aql.eq("$.label", "Individual"))\
.where(aql.and_(aql.eq("source", "tcga"), aql.eq("disease_code", "BRCA")))\
.in_("sampleOf").in_("expressionFor").render(["$.biosampleId", "$.expressions"])
data = {}
for row in c:
    data[row['data'][0]] = row['data'][1]
samples = pandas.DataFrame(data).transpose()

In [42]:
samples

,ENSG00000000003.13,ENSG00000000005.5,ENSG00000000419.11,ENSG00000000457.12,ENSG00000000460.15,ENSG00000000938.11,ENSG00000000971.14,ENSG00000001036.12,ENSG00000001084.9,ENSG00000001167.13,...,ENSG00000281903.1,ENSG00000281904.1,ENSG00000281905.1,ENSG00000281906.1,ENSG00000281909.1,ENSG00000281910.1,ENSG00000281912.1,ENSG00000281918.1,ENSG00000281920.1,ENSG00000281921.1
002997ef-3b93-443a-80a3-bae63e129d99,6.366955,0.586589,35.453655,3.774370,2.524568,3.823734,5.432521,65.775424,6.841912,15.181490,...,0.506842,NaN,NaN,NaN,NaN,NaN,0.235401,0.099563,0.270521,NaN
0039a298-3369-4da5-9bfd-19792e9632be,12.150553,0.758876,37.088989,3.614072,1.732252,7.244930,15.252508,28.950271,3.586114,8.386787,...,0.053779,NaN,NaN,NaN,NaN,NaN,0.385829,NaN,0.229907,NaN
008dafdd-a3d1-4801-8c0a-8714e2b58e48,14.179096,1.919527,26.448078,4.885382,1.877466,3.462328,9.037279,23.066637,4.124768,9.841693,...,0.086951,0.008945,NaN,NaN,0.069404,NaN,1.304113,0.056461,0.118006,NaN
00b54c99-c26d-423a-9871-2552c1603a9c,6.304872,0.342185,24.464916,6.715377,2.465139,2.648335,4.511500,11.598254,15.538686,22.377843,...,1.970267,NaN,NaN,NaN,0.072585,7.696635,1.385347,3.188587,0.518336,NaN
00f81e98-e5c3-43b2-9757-f769ec17f3f8,3.384552,0.029638,42.880659,5.012253,2.702919,1.497163,5.288211,16.509292,2.792837,18.580927,...,0.096265,NaN,NaN,NaN,NaN,NaN,0.474379,NaN,0.171028,NaN
0126489d-c12a-4651-8700-d7090732d7ab,6.527617,0.071031,21.493988,8.221932,2.461373,1.641230,2.553354,36.996732,13.773471,16.011257,...,0.085393,NaN,NaN,NaN,NaN,0.120125,1.184675,0.052531,0.065875,NaN
018b2eaa-216b-4726-a26e-ac726f4b47d3,6.081077,0.124363,23.954035,4.351837,1.941741,0.607092,4.010240,19.346262,3.504671,22.507605,...,0.014689,NaN,NaN,NaN,0.017587,NaN,4.660213,NaN,NaN,NaN
025eac6d-539e-4e43-a6f7-4a2d2def571d,0.815438,0.068225,19.768111,4.237852,1.124948,1.461468,1.245190,20.284293,4.320591,10.321600,...,0.087296,NaN,NaN,NaN,NaN,NaN,0.942114,0.039243,0.032808,NaN
028490aa-d6f3-4366-821a-7cb2c604eff1,16.028508,4.605182,28.304411,1.937910,0.631809,3.477826,8.003091,17.200369,3.179801,6.477186,...,0.039510,NaN,NaN,NaN,NaN,NaN,1.406797,0.692697,0.193037,NaN
02a0a874-6d4e-4e56-8d74-7952c912fed2,15.149100,0.392324,53.689976,2.313736,8.409914,0.897494,2.108691,19.761825,6.475841,15.361096,...,0.007394,NaN,NaN,NaN,NaN,NaN,0.542283,0.043213,NaN,NaN
